<a href="https://colab.research.google.com/github/geoskimoto/AWDB_SOAP_Request/blob/main/pull_awdb_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import xml.dom.minidom as minidom
import json
import xml.etree.ElementTree as ET
# url = "https://www.wcc.nrcs.usda.gov/awdbWebService/services?WSDL"

import numpy as np
import pandas as pd
import datetime
# https://www.dataquest.io/blog/python-api-tutorial/

In [ ]:
#Example from https://www.youtube.com/watch?v=zZU0JZmIvKA
# payload = {'firstname':'John', 'lastname':'Smith'}
# r = requests.get('http://httpbin.org/get?', params=payload)
# print(r.url)
# print(r.content)
# print(r.text)

# response.text.split(' ')

In [ ]:
# url = "https://www.wcc.nrcs.usda.gov/awdbWebService/services?WSDL"
# response = requests.get(url)
# if response.status_code != 200:
#     print('error {}'.format(response.status_code))
# else:
#     data = json.loads(response.text)
#     print(json.dumps(data, indent=4))

In [ ]:
#From the AWDB workbook:

# public Data[] getPeriodOfRecord(String p_stationTriplet, String p_beginDate,     
#                                 String p_endDate){                
#     Data[] values = m_webService.getData(Arrays.asList(p_stationTriplet),
#         "WTEQ", 1, null, Duration.DAILY, true, p_beginDate, p_endDate, true)
#         .toArray(new Data[0]);
#     return values;
# }

In [162]:
# Define Inputs
BeginDate = '01/01/2020'
EndDate = '08/01/2020'

# Create a dictionaries to store the data
headers = {'Content-type': 'text/soap'}
# current_dictionary = {}
 
# Define Web Service URL
URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

# Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
SOAP_current = '''
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
  <SOAP-ENV:Body>
    <q0:getData>
      <stationTriplets>471:ID:SNTL</stationTriplets>
      <elementCd>WTEQ</elementCd>   
      <ordinal>1</ordinal>
      <duration>DAILY</duration>
      <getFlags>false</getFlags>
      <beginDate>$BEGINDATE$</beginDate>
      <endDate>$ENDDATE$</endDate>
      <alwaysReturnDailyFeb29>true</alwaysReturnDailyFeb29>
    </q0:getData>
  </SOAP-ENV:Body>
</SOAP-ENV:Envelope>

'''.strip()

#other element codes: PREC, WTEQ (Water Equivalent/SWE)
 

# Post SOAP Elements to AWDB Web Service and process results - getData

SOAP_current = SOAP_current.replace("$BEGINDATE$", BeginDate)
SOAP_current = SOAP_current.replace("$ENDDATE$", EndDate)

response_current = requests.post(URL, data=SOAP_current, headers=headers)


In [163]:
response_current.text

'<soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/"><soap:Body><ns2:getDataResponse xmlns:ns2="http://www.wcc.nrcs.usda.gov/ns/awdbWebService"><return><beginDate>2020-01-01 00:00:00</beginDate><duration>DAILY</duration><endDate>2020-08-01 00:00:00</endDate><stationTriplet>471:ID:SNTL</stationTriplet><values>7.2</values><values>8.1</values><values>8.1</values><values>8.1</values><values>8.1</values><values>8.5</values><values>8.6</values><values>8.6</values><values>9.0</values><values>9.4</values><values>9.4</values><values>9.9</values><values>10.1</values><values>10.7</values><values>11.6</values><values>11.6</values><values>11.6</values><values>11.9</values><values>12.0</values><values>12.0</values><values>12.0</values><values>12.1</values><values>12.2</values><values>12.4</values><values>12.7</values><values>12.7</values><values>13.1</values><values>13.2</values><values>13.6</values><values>13.7</values><values>13.8</values><values>14.0</values><values>14.0</values

<bound method Response.iter_lines of <Response [200]>>

In [21]:
xmldoc = minidom.parseString(response_current.text)
# placemarks_current = xmldoc_current.getElementsByTagName('return')

In [22]:
xmldoc

In [ ]:
#Javascript example from website

    # public Data[] getPeriodOfRecord(String p_stationTriplet, String p_beginDate,String p_endDate){                
    #     Data[] values = m_webService.getData(Arrays.asList(p_stationTriplet),
    #         "WTEQ", 1, null, Duration.DAILY, true, p_beginDate, p_endDate, true)
    #         .toArray(new Data[0]);
    #     return values;
    # }

#### Create dictionary from request

In [ ]:
{str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{1:xmldoc.getElementsByTagName('values')[0].firstChild.data}}

{'302:OR:SNTL': {1: '4.3'}}

In [ ]:
val_length = len(xmldoc.getElementsByTagName('values'))
val_length

214

In [164]:
#Using OrderedDict
from collections import OrderedDict

date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")
station = xmldoc.getElementsByTagName('stationTriplet')
values = xmldoc.getElementsByTagName('values')
Date = []
for i in range(0, val_length): 
  date += datetime.timedelta(days=1)
  Date.append(str(date))

Date
# woohoo = {station[0].firstChild.data:OrderedDict({Date[j]:values[j].firstChild.data for j in range(0, val_length)})}

# -------------
# bla = []

# for j in range(0, val_length):
#   bla.append(list(zip(Date[j], values.firstChild.data[j])))

# bla

# [np.c_(Date[j], values[j].firstChild.data for j in range(0, val_length)]

# ----------

woohoo = {station[0].firstChild.data:OrderedDict({Date[j]:values[j].firstChild.data for j in range(0, val_length)})}

# woohoo

In [165]:
np.c_[Date[0], values[0].firstChild.data]

ValueError: ignored

In [ ]:
Date[0]

'2020-01-02'

In [ ]:
int(values[0].firstChild.data)

ValueError: ignored

In [ ]:
int(float(values[0].firstChild.data))

4

In [ ]:
# woohoo['302:OR:SNTL']

In [ ]:
# #using zip

# date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")

# Date = []
# for i in range(0, val_length): 
#   date += datetime.timedelta(days=1)
#   Date.append(str(date))

# data2 = {station[0].firstChild.data: list(zip((Date[j],values[j].firstChild.data) for j in range(0, val_length)))}

# # list(zip((Date[j],values[j].firstChild.data) for j in range(0, val_length)))



In [ ]:
date

datetime.datetime(2020, 1, 6, 0, 0)

In [23]:
# import datetime

# date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")

# Date = []
# for i in range(5): 
#   date += datetime.timedelta(days=1)
#   Date.append(str(date))

#     # date += datetime.timedelta(days=1)
#     # print(date) 

In [ ]:
Date

['2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06']

In [ ]:
datetime.

datetime.datetime(2003, 8, 1, 12, 4, 5)

#### Create pd.DataFrame from request

In [166]:
val_length = len(xmldoc.getElementsByTagName('values'))

data = pd.DataFrame([xmldoc.getElementsByTagName('values')[i].firstChild.data for i in range(0,val_length)])

In [167]:
data

,0
0,4.3
1,5.1
2,5.3
3,5.4
4,5.5
...,...
209,0.0
210,0.0
211,0.0
212,0.0


In [168]:
date = datetime.datetime.strptime(BeginDate, "%d/%m/%Y").date() #.strftime("%Y,%m,%d")

Date = []
for i in range(val_length): 
  date += datetime.timedelta(days=1)
  Date.append(str(date))

# {str(xmldoc.getElementsByTagName('stationTriplet')[0].firstChild.data):{Date[j]:xmldoc.getElementsByTagName('values')[j].firstChild.data} for j in range(3)}

In [169]:
type(data.columns[0])

int

In [170]:
data['Date'] = Date
data.columns = ['SWE','Date']
# data.rename(columns={0:'SWE'}) #doesn't work for some reason, so renaming with just data.columns
data.set_index('Date', inplace=True)
data.tail(7)

,SWE
Date,
2020-07-27,0.0
2020-07-28,0.0
2020-07-29,0.0
2020-07-30,0.0
2020-07-31,0.0
2020-08-01,0.0
2020-08-02,0.0


In [171]:
data.head(5)

,SWE
Date,
2020-01-02,4.3
2020-01-03,5.1
2020-01-04,5.3
2020-01-05,5.4
2020-01-06,5.5


## Everything combined as a single function

In [157]:
import requests
import xml.dom.minidom as minidom
import json
import xml.etree.ElementTree as ET
# url = "https://www.wcc.nrcs.usda.gov/awdbWebService/services?WSDL"

import numpy as np
import pandas as pd
import datetime
# https://www.dataquest.io/blog/python-api-tutorial/


def SOAP_Call(stationtriplets, BeginDate, EndDate):

  # Define Inputs
  # BeginDate = '01/01/2020'
  # EndDate = '08/01/2020'

  # Create a dictionaries to store the data
  headers = {'Content-type': 'text/soap'}
  # current_dictionary = {}
  
  # Define Web Service URL
  URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

  # Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
  SOAP_current = '''
  <?xml version="1.0" encoding="UTF-8"?>
  <SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <SOAP-ENV:Body>
      <q0:getData>
        <stationTriplets>$stationTriplets$</stationTriplets>
        <elementCd>WTEQ</elementCd>   
        <ordinal>1</ordinal>
        <duration>DAILY</duration>
        <getFlags>false</getFlags>
        <beginDate>$BEGINDATE$</beginDate>
        <endDate>$ENDDATE$</endDate>
        <alwaysReturnDailyFeb29>true</alwaysReturnDailyFeb29>
      </q0:getData>
    </SOAP-ENV:Body>
  </SOAP-ENV:Envelope>

  '''.strip()

  #other element codes: PREC, WTEQ (Water Equivalent/SWE)
  

  # Post SOAP Elements to AWDB Web Service and process results - getData
  SOAP_current = SOAP_current.replace("$stationTriplets$", stationtriplets)
  SOAP_current = SOAP_current.replace("$BEGINDATE$", BeginDate)
  SOAP_current = SOAP_current.replace("$ENDDATE$", EndDate)

  response_current = requests.post(URL, data=SOAP_current, headers=headers)

  #Parse xml based on tags
  xmldoc = minidom.parseString(response_current.text)

  #Determine number of measurements/days
  val_length = len(xmldoc.getElementsByTagName('values'))

  #Stick parsed values into a dataframe
  data = pd.DataFrame([xmldoc.getElementsByTagName('values')[i].firstChild.data for i in range(0,val_length)])
  #Add dates to the values
  # date = datetime.datetime.strptime(BeginDate, "%m/%d/%Y").date() #.strftime("%Y,%m,%d")
  # Date = []
  # for i in range(0,val_length): 
  #   date += datetime.timedelta(days=1)
  #   Date.append(str(date))
  # data['Date'] = Date
  # # data.columns = ['WTEQ','Date']
  # # data.rename(columns={0:'SWE'}) #doesn't work for some reason, so renaming with just data.columns
  # # data.set_index('Date', inplace=True)
  # return data

In [160]:
data = SOAP_Call('3471:ID:SNTL','06/01/2016','08/01/2019')

In [161]:
data

In [151]:
# x = ['1','2']

data2 = pd.DataFrame(list(zip(data.index, list(map(lambda x: float(x),data.iloc[:,1])))))#,columns=['Date', 'WTEQ'])
# data2.set_index('Date')

IndexError: ignored

In [150]:
data2.plot()

TypeError: ignored

In [109]:
data.tail()

,SWE
Date,
2020-07-29,0.0
2020-07-30,0.0
2020-07-31,0.0
2020-08-01,0.0
2020-08-02,0.0


In [67]:
len(data)

520